In [1]:
!pip -q install langchain tiktoken chromadb pypdf transformers InstructorEmbedding sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.3/249.3 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 96.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.1/965.1 kB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip -q install accelerate bitsandbytes sentencepiece Xformers google.generativeai tensorflow_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.3/113.3 kB 6.8 MB/s eta 0:00:00


In [26]:
pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.1 MB/s eta 0:00:00


In [27]:
from langchain.llms import OpenAI
import openai

In [28]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [29]:
llm = OpenAI(model_name="text-davinci-003", n=2,temperature=0.1, best_of=2)

In [7]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

In [8]:
# Load and process the text files
loader = DirectoryLoader('/content/', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [9]:
len(documents)

217

In [10]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [11]:
len(texts)

673

In [12]:
texts[0]

Document(lc_kwargs={'page_content': '_______________ \n   \nJanuary 27–28, 2009\n1 of 267Meeting of the Federal Op en Market Committee on \nJanuary 27 –28, 2009 \nA joint meeting of the Federal Open Market  Committee and Board of Governors of the \nFederal Reserve System was held in the offices of the Board of Governors in Washington, D.C., on Tuesday, January 27, 2009, at 1:30 p.m. a nd continued on Wednesday, January 28, 2009, at \n9:00 a.m.  Those present were the following: \nMr. Bernanke, Chairman \nMr. Dudley, Vice Chairman \nMs. Duke \nMr. Evans \nMr. Kohn \nMr. Lacker \nMr. Lockhart \nMr. Warsh \nMs. Yellen\n \nMr. Bullard, Ms. Cumming, Mr. Hoenig, Ms. Pianalto, and Mr. Rosengren, Alternate \nMembers of the Federal Open Market Committee Messrs. Fisher, Plosser, and Stern, Presidents  of the Federal Reserve Banks of Dallas, \nPhiladelphia, and Minneapolis, respectively \nMr. Madigan, Secretary and Economist', 'metadata': {'source': '/content/FOMC20090128meeting.pdf', 'page': 0}}

In [13]:
from langchain.embeddings import TensorflowHubEmbeddings
url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
tf = TensorflowHubEmbeddings(model_url=url)

In [14]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = tf

vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [15]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [16]:
# Now we can load the persisted database from disk, and use it as normal. 
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

In [17]:
retriever = vectordb.as_retriever()

In [18]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [30]:
# create the chain to answer questions 
qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [20]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [31]:
# full example
query = "What are the major topics discussed in the meeting?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The major topics discussed in the meeting include the effectiveness of the modified meeting structure, the
possibility of two-day meetings, the status of adopting a numerical objective for inflation, and the views of
the political bodies on going forward with an explicit objective.


Sources:
/content/FOMC20090128meeting.pdf
/content/FOMC20090128meeting.pdf
/content/FOMC20090128meeting.pdf


In [32]:
# full example
query = "What is the sentiment around inflation in the meeting? Can you give a score as well"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The sentiment around inflation in the meeting is that it has fallen significantly and is likely to be low in
the medium term, but there is some risk that it could fall below levels consistent with good economic
performance and there is concern about a deflationary trap at the global level. Maintaining stable inflation
expectations is seen as very important.


Sources:
/content/FOMC20090128meeting.pdf
/content/FOMC20090128meeting.pdf
/content/FOMC20090128meeting.pdf


In [34]:
# full example
query = "What is the sentiment around inflation in the meeting? Can you give a score as well? The score should be from -1 to 1. 1 being highly positive and -1 being highly negative."
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The sentiment around inflation in the meeting is slightly negative. The score would be -0.5.


Sources:
/content/FOMC20090128meeting.pdf
/content/FOMC20090128meeting.pdf
/content/FOMC20090128meeting.pdf
